In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Chargement des données
try:
    df = pd.read_csv('./train.csv')
    print("Données chargées avec succès.")
except FileNotFoundError:
    print("Erreur : Le fichier 'train.csv' est introuvable.")

Données chargées avec succès.


In [2]:
# 1. Suppression des colonnes non informatives pour le modèle
df = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

# 2. Nettoyage des lignes avec des valeurs manquantes critiques (hors Age)
df = df.dropna(subset=['Embarked', 'Fare'])

# 3. Encodage : Sexe (0: Homme, 1: Femme)
df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})

# 4. Encodage : Embarked (Création de variables dummy)
df = pd.get_dummies(df, columns=['Embarked'], drop_first=True)

In [3]:
# --- Préparation des données pour le clustering ---
# On exclut 'Survived' (le target) et 'Age' (ce qu'on cherche) des features de clustering
features_clustering = [col for col in df.columns if col not in ['Age', 'Survived']]

df_age_connu = df[df['Age'].notnull()].copy()
df_age_manquant = df[df['Age'].isnull()].copy()

# --- Mise à l'échelle (Standardisation) ---
scaler = StandardScaler()
X_connu_scaled = scaler.fit_transform(df_age_connu[features_clustering])
X_manquant_scaled = scaler.transform(df_age_manquant[features_clustering])

# --- Entraînement du K-Means ---
K_optimal = 10  # Vous pouvez ajuster ce nombre
kmeans = KMeans(n_clusters=K_optimal, random_state=42, n_init=10)
df_age_connu['Cluster'] = kmeans.fit_predict(X_connu_scaled)

# --- Calcul des moyennes d'âge par cluster ---
moyenne_age_par_cluster = df_age_connu.groupby('Cluster')['Age'].mean()
print("Moyennes d'âge par cluster calculées.")

# --- Prédiction et Imputation ---
clusters_predits = kmeans.predict(X_manquant_scaled)
df_age_manquant['Age'] = [moyenne_age_par_cluster[c] for c in clusters_predits]

# --- Réassemblage du DataFrame final ---
df_final = pd.concat([df_age_connu.drop(columns='Cluster'), df_age_manquant])
df_final = df_final.sort_index()

print(f"Imputation terminée. Valeurs manquantes restantes : {df_final['Age'].isnull().sum()}")

c:\Users\louis\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1446: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  warnings.warn(


Moyennes d'âge par cluster calculées.
Imputation terminée. Valeurs manquantes restantes : 0


In [4]:
df_final

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S
0,0,3,0,22.000000,1,0,7.2500,False,True
1,1,1,1,38.000000,1,0,71.2833,False,False
2,1,3,1,26.000000,0,0,7.9250,False,True
3,1,1,1,35.000000,1,0,53.1000,False,True
4,0,3,0,35.000000,0,0,8.0500,False,True
...,...,...,...,...,...,...,...,...,...
886,0,2,0,27.000000,0,0,13.0000,False,True
887,1,1,1,19.000000,0,0,30.0000,False,True
888,0,3,1,25.947073,1,2,23.4500,False,True
889,1,1,0,26.000000,0,0,30.0000,False,False
